In [ ]:
import os, re, time, random
from dataclasses import dataclass, field
from typing import List, Dict
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(r"C:\my_envs\.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_NAME = os.getenv("MODEL_NAME", "gpt-4o-mini")

client = OpenAI(api_key=OPENAI_API_KEY)

TURN_MINUTES = 10
ELAPSED_END_MINUTES = 240

@dataclass
class GameState:
    hour: int = 23
    minute: int = 0
    loc: str = "거실"
    hp: int = 3
    cur: int = 0
    items: List[str] = field(default_factory=list)
    clues: List[str] = field(default_factory=list)
    flags: Dict[str, bool] = field(default_factory=lambda: {
        "간식장_잠금": True,
        "서랍_열림": False,
        "간식장_열림": False,
        "집사_참치캔_따줌": False,
    })
    last_choice: str = ""
    elapsed_min: int = 0

    def advance_time(self, minutes=TURN_MINUTES):
        self.elapsed_min += minutes
        total = self.hour * 60 + self.minute + minutes
        self.hour, self.minute = divmod(total, 60)
        if self.hour >= 24:
            self.hour -= 24

def fmt_state(s: GameState) -> str:
    t = f"{s.hour:02d}:{s.minute:02d}"
    items = ",".join(s.items) if s.items else "-"
    clues = ",".join(s.clues) if s.clues else "-"
    flags = ",".join([f"{k}:{'true' if v else 'false'}" for k,v in s.flags.items()])
    return (
        "[STATE]\n"
        f"TIME={t}\n"
        f"LOC={s.loc}\n"
        f"HP={s.hp}\n"
        f"CUR={s.cur}\n"
        f"ITEMS=[{items}]\n"
        f"CLUES=[{clues}]\n"
        f"FLAGS={{ {flags} }}\n"
        "[/STATE]\n"
    )

with open(r"C:\AIcamp\practice\샛별이의 참치찾기_Prompt.txt", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()

USER_TURN_PROMPT = """다음 상태와 최근 행동을 반영해 한 턴을 진행해.
{STATE}
이전입력: {PLAYER_INPUT}
"""

def llm(system_msg: str, user_msg: str) -> str:
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role":"system","content":system_msg},
            {"role":"user","content":user_msg}
        ],
        temperature=0.7,
        max_tokens=400,
    )
    return resp.choices[0].message.content

def shallow_update_from_scene(s: GameState, text: str):
    txt = text

    if "참치" in txt and "참치냄새" not in s.clues:
        s.clues.append("참치냄새")

    if "열쇠" in txt and "열쇠" not in s.items:
        s.items.append("열쇠")

    if "청소기" in txt and s.hp > 0 and random.random() < 0.5:
        s.hp -= 1

    for room in ["부엌", "거실", "안방", "서재", "현관", "베란다"]:
        if room in txt:
            s.loc = room
            break

    if "창가" in txt:
        s.cur = min(5, s.cur + 1)

    if ("간식장" in txt or "찬장" in txt) and ("열" in txt or "연다" in txt or "열어" in txt):
        if "열쇠" in s.items:
            s.flags["간식장_잠금"] = False
            s.flags["간식장_열림"] = True
            if "참치캔" not in s.items:
                s.items.append("참치캔")

    can_open_verbs = ["따", "따줘", "따달", "열어", "오픈", "따준다", "따준다", "열어준다"]
    if ("집사" in txt) and any(v in txt for v in can_open_verbs):
        if "참치캔" in s.items and s.flags.get("간식장_열림", False):
            s.flags["집사_참치캔_따줌"] = True

def check_ending(s: GameState):
    if s.elapsed_min >= ELAPSED_END_MINUTES:
        return "B: 새벽 고요 엔딩" if s.hp > 0 else "C: 낮잠 엔딩"
    if s.hp <= 0:
        return "C: 낮잠 엔딩"

    if s.loc == "부엌" and s.flags.get("집사_참치캔_따줌", False):
        if s.elapsed_min <= 160:
            return "S: 전설의 참치캔 엔딩"
        return "A: 특제 간식 획득 엔딩"

    return None

def main():
    print("=== 샛별이 탐험 RPG 🐾 ===")
    state = GameState()
    player_input = "게임 시작"

    while True:
        ending = check_ending(state)
        if ending:
            print("\n[엔딩] " + ending)
            break

        user_msg = USER_TURN_PROMPT.format(
            STATE=fmt_state(state),
            PLAYER_INPUT=player_input
        )
        out = llm(SYSTEM_PROMPT, user_msg)

        print("\n" + out.strip() + "\n")

        m = re.search(r"\[SCENE\](.*?)(?=\[CHOICES\]|\[/TURN\])", out, re.S)
        if m:
            scene_text = m.group(1).strip()
            shallow_update_from_scene(state, scene_text)

        user_choice_text = input("입력(예: 부엌으로 간다 / 간식장을 연다 / 집사를 깨워 참치캔 따달라고 조른다, q=종료): ").strip()
        if user_choice_text.lower() == "q":
            print("종료합니다. 잘 자요, 샛별이! 😴")
            break

        player_input = user_choice_text
        shallow_update_from_scene(state, user_choice_text)

        state.advance_time()
        time.sleep(0.2)

if __name__ == "__main__":
    main()





=== 샛별이 탐험 RPG 🐾 ===

[STATE]
TIME=23:10
LOC=거실
HP=3
CUR=0
ITEMS=[-]
CLUES=[-]
FLAGS={ 간식장_잠금:true,서랍_열림:false,간식장_열림:false,집사_참치캔_따줌:false }

거실은 조용하고, 먼지 냄새가 감돈다. 나는 조심스레 주변을 살펴본다. 바닥은 차갑고, 발바닥이 미끄러질까 걱정이다. 

“무언가 흥미로운 걸 찾아야겠다...냐옹.” 

내 시선은 거실의 구석으로 향한다. 로봇청소기가 묵묵히 제자리에서 대기 중이다. 혹시 이걸로 뭔가 할 수 있을까? 

1. 로봇청소기를 만져본다.
2. 거실의 다른 구석을 탐색한다.
3. 집사에게 다가가 본다. 

어떤 행동을 선택할래?


부엌에 들어서자, 참치의 고소한 냄새가 코를 자극한다. 바닥은 차가운 타일로, 발이 미끄러질 듯하다. 간식장 앞에 서면, 잠금 장치가 반짝인다...냐옹. 

주변을 살피니 로봇청소기가 조용히 움직이고 있다. 소음이 없어서 다행이다. 더 깊이 들어가 볼까, 아니면 다른 방으로 나갈까 고민된다.

1. 간식장 잠금을 해제해 볼까?
2. 로봇청소기를 피해서 서재로 가볼까? 

어떤 선택을 할래?


부엌의 차가운 타일 바닥에서 발을 디디며, 간식장 쪽으로 다가갔어. 콧속에 참치의 고소한 향이 스며들고, 냉장고의 웅웅거리는 소리가 귀에 맴돌아. 하지만 간식장은 잠겨 있어. 

“어떻게 열 수 있을까...냐옹?” 귀가 쫑긋, 주위를 살피며 다른 방법을 찾아봐야겠어.

1. 서랍을 열어보러 가기
2. 로봇청소기를 다가가게 해보기
3. 집사에게 소리내어 깨워보기

어떤 행동을 할래?


[STATE]
TIME=23:40
LOC=부엌
HP=3
CUR=0
ITEMS=[-]
CLUES=[-]
FLAGS={ 간식장_잠금:true,서랍_열림:true,간식장_열림:false,집사_참치캔_따줌:false }
[/STATE]

서랍을 열자, 차가운 금속 소리가 귀에 맴돈다. 서랍 안에는 낡은 종이와 작은 장난감 쥐가 있다. 종이는 